In [1]:
%pwd

'd:\\Repos\\text-Summarizer-Project\\research'

In [2]:
import os

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Repos\\text-Summarizer-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories


In [12]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return  data_ingestion_config

In [13]:
import os 
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [14]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
            url = self.config.source_URL,
            filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [16]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-02-23 23:36:28,198: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-23 23:36:28,202: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-23 23:36:28,203: INFO: common: Created directory at: artifacts]
[2025-02-23 23:36:28,204: INFO: common: Created directory at: artifacts/data_ingestion]
[2025-02-23 23:36:30,222: INFO: 248448346: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 2945116
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "d15c1b24ab673d07140382c9221cb271724917a3f6a160c86c7e8cc0610fe0ae"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 3298:11F613:11A304:36DA5E:67BB63A4
Accept-Ranges: bytes
Date: Sun, 23 Feb 2025 18:06:29 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10237-MAA
X-Cache: MI